## Read Smallcases

In [1]:
import numpy as np
import pandas as pd

In [36]:
id = "0025"
smallcase = pd.ExcelFile(f"data/smallcases/SCNM_{id}_Timeline.xlsx")

constituents = pd.read_excel(smallcase, sheet_name="Historical Constituents")
constituents.head()

,Date Range,Constituents,Weightage
0,2017-11-15 to 2017-11-30,JBM Auto Ltd,0.08
1,NaN,Igarashi Motors India Ltd,0.08
2,NaN,Mahindra and Mahindra Ltd,0.08
3,NaN,Tata Motors Ltd,0.08
4,NaN,Hero MotoCorp Ltd,0.08


In [37]:
indexes = pd.read_excel(smallcase, sheet_name="Historical Index Values")
indexes.head()

,Date,Electric Mobility,NIFTY Large & Midcap,Rebalance Occured
0,2017-11-15,100.00,100.00,NaN
1,2017-11-16,101.03,101.08,NaN
2,2017-11-17,101.62,101.91,NaN
3,2017-11-20,102.26,102.32,NaN
4,2017-11-21,102.52,102.55,NaN


### Map and POST Smallcase Name to Upstox Instrument

In [23]:
# Connect to db
import os
import sys

backend_path = os.path.abspath(os.path.join(os.getcwd(), '../backend'))
sys.path.append(backend_path)
from app.internal.firebase import db

In [42]:
postable_constituents = constituents["Constituents"].unique()
len(postable_constituents)

30

In [95]:
async def query_ticker_name(name: str):
    return await db.collection("tickers").where("name", ">=", name).where("name", "<=", name + '\uf8ff').get()

In [96]:
successful_constitents = []
failed_constitents = []


for c in postable_constituents:
    q = c.upper()
    docs = await query_ticker_name(q)
    if len(docs) == 0:
        failed_constitents.append(c)
    else:
        for e in docs:
            await db.collection("tickers").document(e.id).update({
                "smallcase_name": c
            })
            successful_constitents.append((c, e.id))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:302: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/var/folders/4x/3kzzlsjs1q36v35r81960pt40000gn/T/ipykernel_64588/2276505340.py:2: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return await db.collection("tickers").where("name", ">=", name).where("name", "<=", name + '\uf8ff').get()


In [97]:
len(successful_constitents)


10

10 successful, 20 failed, wrangle and try again

In [108]:
# All return upper

def replace_ltd(s):
    return s.upper().replace("LTD", "LIMITED")

def replace_technologies(s):
    return s.upper().replace("TECHNOLOGIES", "TECHNO.")

def replace_company(s):
    return s.upper().replace("COMPANY", "CO")

def replace_corporation(s):
    return s.upper().replace("CORPORATION", "CORP")

In [109]:
async def post_smallcase_name(id: str, name: str, s: list, f: list):
    await db.collection("tickers").document(id).update({
                "smallcase_name": name
            })
    s.append((name, id))
    f.remove(name)

In [113]:
# Ltd -> Limited
for f in failed_constitents:
    q = replace_ltd(f)
    docs = await query_ticker_name(q)
    if len(docs) != 0:
        for e in docs:
            post_smallcase_name(e.id, f, successful_constitents, failed_constitents)

/var/folders/4x/3kzzlsjs1q36v35r81960pt40000gn/T/ipykernel_64588/2276505340.py:2: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return await db.collection("tickers").where("name", ">=", name).where("name", "<=", name + '\uf8ff').get()


In [114]:
len(successful_constitents), len(failed_constitents)

(19, 11)

Found 9 new equities

In [112]:
failed_constitents

['Mahindra and Mahindra Ltd',
 'Indian Oil Corporation Ltd',
 'Himadri Speciality Chemical Ltd',
 'Hindalco Industries Ltd',
 'Amara Raja Energy & Mobility Ltd',
 'Power Grid Corporation of India Ltd',
 'Endurance Technologies Ltd',
 'Bharat Heavy Electricals Ltd',
 'Tata Power Company Ltd',
 'Samvardhana Motherson International Ltd',
 'Sona BLW Precision Forgings Ltd']

In [116]:
nse = pd.read_csv("data/upstox/NSE.csv")
equities = nse.query('instrument_type == "EQUITY" and exchange == "NSE_EQ"')
equities.head()

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
77,NSE_EQ|DUMMYSAN005,14747.0,011NSETEST,011NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
78,NSE_EQ|DUMMYSAN006,14751.0,021NSETEST,021NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
79,NSE_EQ|DUMMYSAN007,14753.0,031NSETEST,031NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
80,NSE_EQ|DUMMYSAN008,14755.0,041NSETEST,041NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
81,NSE_EQ|DUMMYSAN009,14758.0,051NSETEST,051NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ


In [126]:
import difflib
df = equities.copy(deep=True)
all_matches = pd.DataFrame() 
r = {}
for f in failed_constitents:
    df['similarity'] = df["name"].apply(lambda x: difflib.SequenceMatcher(None, x.lower(), f.lower()).ratio())
        
    # Sort by similarity and get top 3 matches
    top_match = df.sort_values(by='similarity', ascending=False).head(1)
    top_match['smallcase_name'] = f
        
    # Append the top match to the all_matches DataFrame
    all_matches = pd.concat([all_matches, top_match], ignore_index=True)

all_matches.head()

,similarity,smallcase_name
0,0.916667,Mahindra and Mahindra Ltd
1,0.851064,Indian Oil Corporation Ltd
2,0.892857,Himadri Speciality Chemical Ltd
3,0.958333,Hindalco Industries Ltd
4,0.877193,Amara Raja Energy & Mobility Ltd
